In [1]:
from z3 import *

import numpy as np
from itertools import combinations
from utils import *
import math
import time

In [15]:
instance_n=2 #from 1 to 21

if instance_n<10:
    file_name='inst0'+str(instance_n)+'.dat'
else:
    file_name='inst'+str(instance_n)+'.dat'

file = open('./Instances/'+file_name, 'r')

splitted_file = file.read().split('\n')

m = int(splitted_file[0])
n = int(splitted_file[1])
cpt = list(map(int, splitted_file[2].split(' ')))
tmp_sz=splitted_file[3].split(' ')
if '' in tmp_sz:
    sz=list(map(int, [tmp_sz[i] for i in range(len(tmp_sz)) if tmp_sz[i]!='']))
else:
    sz = list(map(int, splitted_file[3].split(' ')))
D = [list(map(int, line.strip().split(' '))) for line in splitted_file[4:(n+5)]]
print(n)
print(m)


if n%m!=0:
    remaining_item=n+(m-n%m)
else:
    remaining_item=n
remaining_cour=m
n_x_cour=remaining_item//m
item_mean_weight=np.mean(sz)
w_x_cour = item_mean_weight * n_x_cour

cpt_1=copy.deepcopy(cpt)
cpt_1.sort()
for i in range(m-1):
    #print(remaining_cour, remaining_item)
    if (remaining_item%remaining_cour)!=0:
        n_x_cour=remaining_item//remaining_cour+1
    else:
        n_x_cour=remaining_item//remaining_cour
    remaining_cour-=1
    if cpt_1[i]>=item_mean_weight*n_x_cour:
        #print('ok')
        remaining_item-=n_x_cour
    else:
        this_cour_it = cpt_1[i]//item_mean_weight
        remaining_item-=this_cour_it
    print('n_x', n_x_cour)
print(n_x_cour)




print(np.mean([np.mean(i[:-1]) for i in D[:-1]])  *(n/m) +(np.min( [np.mean(D[n][:-1]), np.mean([D[i][n] for i in range(n)]) ] )  * 2*m))
print(np.mean([np.mean(i) for i in D])  *(n/m) +(np.mean( [np.mean(D[n][:-1]), np.mean([D[i][n] for i in range(n)]) ] )  * m))
# NO print(np.mean([np.mean(i) for i in D])  *(n//m +1) +(np.mean( [np.min(D[n][:-1]), np.min([D[i][n] for i in range(n)]) ] )  * m))
# no 9 print(np.mean([np.mean(i) for i in D])  *(n//m +1) +(np.max( [np.min(D[n][:-1]), np.min([D[i][n] for i in range(n)]) ] )  * m))

#no :( print(np.mean([np.mean(i) for i in D])  *n_x_cour +(np.mean( [np.mean(D[n][:-1]), np.min([D[i][n] for i in range(n)]) ] )  * m))




9
6
n_x 2
n_x 2
n_x 2
n_x 2
n_x 2
2
1099.4814814814813
613.29


In [3]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [4]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [5]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [6]:
def print_items_for_each_courier(couriers_items, items_predecessors, sz, cpt):
    # Creare un dizionario per mappare ogni corriere ai suoi pacchi
    couriers_dict = {}
    for i, j in couriers_items:
        if i not in couriers_dict:
            couriers_dict[i] = []
        couriers_dict[i].append(j)
    #print('cour_dict', couriers_dict)

    n=0
    for value in couriers_dict.values():
        n = n+len(value)
    #print(n)


    # Creare un altro dizionario per mappare ogni pacco al suo predecessore
    
    predecessors_dict = {i: j for j, i in items_predecessors}
    #print('pred_dict', predecessors_dict)
    
    # Per ogni corriere, seguire la catena di predecessori per ottenere l'ordine corretto dei pacchi
    for courier, items in couriers_dict.items():
        print(f"Corriere {courier}: ", end='')
        item = n + courier
        order = [item]
        while item in predecessors_dict and predecessors_dict[item] not in order:
            item = predecessors_dict[item]
            order.append(item)
        item = predecessors_dict[item]
        order.append(item)
        print(" -> ".join(map(str, order)), end='   ')
        print('Peso trasportato: ', end='')
        weight=0
        for i in couriers_dict[courier]:
            weight += sz[i]

        print(weight, '/', cpt[courier])

In [7]:
def max_z3(vec):
  m = vec[0]
  for value in vec[1:]:
    m = If(value > m, value, m)
  return m

In [16]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes (len = n)
   cpt array of couriers' capacities (len = m)
'''
def mcp(n, m, D, sz, cpt):

   #create Solver
   solv= Solver()

   '''
      vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   ''' 
   #create a [m x (n+m)] matrix v. v[i][j]==True if the i-th courier takes the j-th item (or starting/ending point)
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

   for j in range(m):
      for i in range(m):
         if j==i:
            solv.add(v[i][n+j])
         else:
            solv.add(Not(v[i][n+j]))
   
   
   #Constraint peso
   for c in range(m):
      solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+m):
      col_i = []
      for j in range(n+m):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   #Constraint coerenza courier - predecessore/successore
   for c in range(m):
      solv.add(And(  [Implies(And([ v[c][i], pred[i][j]] ) , v[c][j]) for i in range(n+m) for j in range(n+m)]  ))
   
   '''
      no_loops 
   '''
   #no_loops = IntVector('no_loops', n)
   no_loops = [BitVec(f'no_loop{i}', 16) for i in range(n)]
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   dist_vector = IntVector('dist_vector', m)
   for courier in range(m):
      starting_point = [If(pred[item][n+courier], 1, 0)*D[n][item] for item in range(n)]
      ending_point = [If(pred[n+courier][item], 1, 0)*D[item][n] for item in range(n)]
      mid_points = [If(And(v[courier][i], pred[i][j]), 1, 0)*D[j][i] for i in range(n) for j in range(n)]
      distance_of_this_path = Sum(starting_point + mid_points + ending_point)
      solv.add(dist_vector[courier] == distance_of_this_path)
   max_dist = Int('max_dist')
   #max_dist = max_z3(dist_vector)
   solv.add(max_dist == max_z3(dist_vector))
   
   return solv, max_dist, pred, v


In [10]:
def mcp_small(n, D):

   #create Solver
   solv= Solver()
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+1)]for i in range(n+1)]

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+1):
      col_i = []
      for j in range(n+1):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))
   
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(pred[i][j], no_loops[i]>no_loops[j]) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)

   #Distance
   
   starting_point = [If(pred[item][n], 1, 0)*D[n][item] for item in range(n)]
   ending_point = [If(pred[n][item], 1, 0)*D[item][n] for item in range(n)]
   mid_points = [If(pred[i][j], 1, 0)*D[j][i] for i in range(n) for j in range(n)]
   distance_of_this_path = Sum(starting_point + mid_points + ending_point)
   max_dist = Int('max_dist')
   solv.add(max_dist == distance_of_this_path)
   
   return solv, max_dist, pred


In [11]:
def mcp_3D(n, m, D, sz, cpt):

   #create Solver
   solv= Solver()

   '''
      cour_pred matrix  
   '''
   #courier_pred[c][i][j] = true if the j-th item is the predecessor of the i-th item and they are taken by the courier c.
   cour_pred = [[[Bool(f"cour({c})_predOf({i})_is{j}") for i in range(n+1)] for j in range(n+1)] for c in range(m)]

   #each pack have to be carried by exactly one courrier
   for i in range(n):
      solv.add(exactly_one_seq([cour_pred[c][i][j] for j in range(n+1) for c in range(m)], f"v11_{i}"))
   for j in range(n):
      solv.add(exactly_one_seq([cour_pred[c][i][j] for i in range(n+1) for c in range(m)], f"v22_{j}"))
   for c in range(m):
      solv.add(exactly_one_seq([cour_pred[c][n][j] for j in range(n+1)], f"v13_{c}"))
      solv.add(exactly_one_seq([cour_pred[c][i][n] for i in range(n+1)], f"v41_{c}"))
   print('cour pred fatta')

   #Constraint peso   
   for c in range(m):
      solv.add(Sum([If(cour_pred[c][i][j], 1, 0)*sz[i] for j in range(n+1) for i in range(n)]) <= cpt[c])
   print('peso fatto')
   #Constraint coerenza courier - predecessore/successore
   for c in range(m):
      solv.add(And(  [Implies(cour_pred[c][i][j] , Or([cour_pred[c][j][k] for k in range(n+1)])) for i in range(n) for j in range(n)]  ))
   print('coerenza fatta')
   '''
      no_loops 
   '''
   no_loops = IntVector('no_loops', n)
   solv.add(And([Implies(cour_pred[c][i][j], no_loops[i]>no_loops[j]) for c in range(m) for i in range(n) for j in range(n)]))
   #solv.add(no_loops[0]==1)
   print('no loop fatto')
   #Distance
   dist_vector = IntVector('dist_vector', m)
   for c in range(m):
      starting_point = [If(cour_pred[c][i][n], 1, 0)*D[n][i] for i in range(n+1)]
      ending_point = [If(cour_pred[c][n][j], 1, 0)*D[j][n] for j in range(n)]
      mid_points = [If(cour_pred[c][i][j], 1, 0)*D[j][i] for i in range(n) for j in range(n)]
      distance_of_this_path = Sum(starting_point + mid_points + ending_point)
      solv.add(dist_vector[c] == distance_of_this_path)
   max_dist = Int('max_dist')
   #max_dist = max_z3(dist_vector)
   solv.add(max_dist == max_z3(dist_vector))
   print('max dist fatto')
   return solv, max_dist, cour_pred


In [3]:
import numpy as np
import copy

def clustering(D):
    list_of_mins=[]
    n=len(D)-1
    Closest=[]
    for item in range(n): 
        tmp = copy.deepcopy(D[item])
        tmp[item]=1000
        Closest.append(np.argmin(tmp[:-1])) 
        list_of_mins.append(np.min(tmp[:-1]))
    #print(list_of_mins)
    clusters=[]
    already_in_cluster=[]
    for item in range(n):
        if item==Closest[Closest[item]] and (item not in already_in_cluster):
            clusters.append([item, Closest[item]])
            already_in_cluster.append(item)
            already_in_cluster.append(Closest[item])
        elif item!=Closest[Closest[item]]:
            clusters.append([item])
    clusters.append([n])
    D_new=[]
    n_new=len(clusters)-1

    for row in range(n_new): 
        D_new.append([]) 
        for column in range(n_new): 
            if row==column:
                D_new[row].append(0)
            else:
                tmp_dist=[]
                for i in clusters[row]:
                    for j in clusters[column]:
                        tmp_dist.append(D[i][j])
                D_new[row].append(np.mean(tmp_dist))
        tmp_dist=[]
        for i in clusters[row]:
            tmp_dist.append(D[i][n])
        D_new[row].append(np.mean(tmp_dist))
    D_new.append([])

    for column in range(n_new):
        tmp_dist=[]
        for i in clusters[column]:
            tmp_dist.append(D[n][i])
        D_new[n_new].append(np.mean(tmp_dist))
    D_new[n_new].append(0)

    return D_new, clusters

In [14]:
clustering_iterations=4

D_opt=D
clusters=[ [i] for i in range(n+1) ]
for i in range(clustering_iterations):
    D_opt, new_clusters=clustering(D_opt)
    old_clusters=clusters
    clusters=[]
    for clus in new_clusters:
        clusters.append([])
        for item in clus:
            for old_item in old_clusters[item]:
                clusters[-1].append(old_item)
print(clusters)
print(len(D_opt))
print([len(k) for k in clusters])

[[0, 84, 75], [1, 47, 29, 30, 130], [2, 4, 138, 76, 14, 124], [3, 117, 17], [5, 86, 78, 116], [6, 72], [7, 37, 15, 21, 60], [8, 68], [9, 103, 34, 41], [10, 46, 65, 113, 38], [11, 25, 109, 119, 141], [12, 43, 122, 114], [13, 111, 137, 20, 63, 82, 106], [16, 70, 98, 74], [18, 131, 64], [19, 97, 110, 129, 77], [22, 49, 89], [23, 45, 134, 55], [24, 62], [26, 79, 83], [27, 101, 100, 121], [28, 56], [31, 32, 90, 61], [33, 108], [35, 51, 123], [36, 87, 125], [39, 52, 128], [40, 135, 91], [42], [44, 67, 107], [48, 142], [50], [53, 58, 71], [54, 80], [57, 59, 99, 93, 85, 94, 88], [66, 69], [73, 96, 105, 139, 132], [81, 115], [92, 140], [95], [102], [104], [112], [118, 120], [126], [127], [133], [136], [143]]
49
[3, 5, 6, 3, 4, 2, 5, 2, 4, 5, 5, 4, 7, 4, 3, 5, 3, 4, 2, 3, 4, 2, 4, 2, 3, 3, 3, 3, 1, 3, 2, 1, 3, 2, 7, 2, 5, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]


In [15]:
clusters_paths=[]
cluster_path_distances=[]

for cluster in clusters:
    n_cluster = len(cluster)
    if len(cluster)>1:
        D_clus=[]
        for i in cluster:
            D_clus.append([])
            for j in cluster:
                D_clus[-1].append(D[i][j])
            D_clus[-1].append(0)
        D_clus.append([0 for k in range(n_cluster+1)])
        #print(D_clus)
        tmp_solv, max_dist, pred =mcp_small(n = len(cluster), D=D_clus)
        while tmp_solv.check()==sat:
            tmp_model=tmp_solv.model()
            best_dist=tmp_model.evaluate(max_dist).as_long()
            tmp_solv.add(max_dist<best_dist)
        cluster_copy=copy.deepcopy(cluster)
        cluster_copy.append(-1)
        clusters_paths.append([(cluster_copy[i],cluster_copy[j]) for i in range(n_cluster+1) for j in range(n_cluster+1) if tmp_model.evaluate(pred[i][j])])
        cluster_path_distances.append(best_dist)

In [16]:
tmp = [clusters[i] for i in range(len(clusters)) if len(clusters[i])>1]
m = max([len(i) for i in tmp])
print('[', end='')
for i in tmp:
    print('|', end=' ')
    for j in range(m):
        if j<len(i):
            print(i[j], end=', ')
        else:
            print(-1, end=', ')
    print('')
print('|]')
print(len(clusters))
print([len(clusters[i]) for i in range(len(clusters)) if len(clusters[i])>1])

[| 0, 84, 75, -1, -1, -1, -1, 
| 1, 47, 29, 30, 130, -1, -1, 
| 2, 4, 138, 76, 14, 124, -1, 
| 3, 117, 17, -1, -1, -1, -1, 
| 5, 86, 78, 116, -1, -1, -1, 
| 6, 72, -1, -1, -1, -1, -1, 
| 7, 37, 15, 21, 60, -1, -1, 
| 8, 68, -1, -1, -1, -1, -1, 
| 9, 103, 34, 41, -1, -1, -1, 
| 10, 46, 65, 113, 38, -1, -1, 
| 11, 25, 109, 119, 141, -1, -1, 
| 12, 43, 122, 114, -1, -1, -1, 
| 13, 111, 137, 20, 63, 82, 106, 
| 16, 70, 98, 74, -1, -1, -1, 
| 18, 131, 64, -1, -1, -1, -1, 
| 19, 97, 110, 129, 77, -1, -1, 
| 22, 49, 89, -1, -1, -1, -1, 
| 23, 45, 134, 55, -1, -1, -1, 
| 24, 62, -1, -1, -1, -1, -1, 
| 26, 79, 83, -1, -1, -1, -1, 
| 27, 101, 100, 121, -1, -1, -1, 
| 28, 56, -1, -1, -1, -1, -1, 
| 31, 32, 90, 61, -1, -1, -1, 
| 33, 108, -1, -1, -1, -1, -1, 
| 35, 51, 123, -1, -1, -1, -1, 
| 36, 87, 125, -1, -1, -1, -1, 
| 39, 52, 128, -1, -1, -1, -1, 
| 40, 135, 91, -1, -1, -1, -1, 
| 44, 67, 107, -1, -1, -1, -1, 
| 48, 142, -1, -1, -1, -1, -1, 
| 53, 58, 71, -1, -1, -1, -1, 
| 54, 80, -1, -1, -

In [17]:
#Modify D_opt?

n_new = len(clusters)-1

first_items_for_clusters=[]
last_item_for_clusters=[]

i=0
for clus in clusters:
    if len(clus)>1:
        path=clusters_paths[i]
        for couple in path:
            if couple[0]==-1:
                last_item_for_clusters.append(couple[1])
            elif couple[1]==-1:
                first_items_for_clusters.append(couple[0])
        i+=1
    else:
        last_item_for_clusters.append(clus[0])
        first_items_for_clusters.append(clus[0])

D_new=[]
for i in range(n_new+1):
    D_new.append([])
    for j in range(n_new+1):
        if j==i:
            D_new[-1].append(0)
        else:
            D_new[-1].append( D[first_items_for_clusters[i]][last_item_for_clusters[j]] )

In [18]:
sz_new=[]
for item in range(n_new):
    sz_new.append(sum([sz[i] for i in clusters[item]]))

In [18]:
starting_time = time.time()
timeout = starting_time + 60*4
check_timeout = timeout-time.time()

solv, max_dist, pred, v = mcp(n_new, m, D_new, sz_new, cpt)
print('builded')
solv.add(max_dist<1354-sum(cluster_path_distances))
to_print=[check_timeout]

'''
if checked_solv==sat:
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('solution 1) obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    #solv.add( Not( And([pred[i][j] for i in range(n) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    solv.add( max_dist < best_distance)
    print('qui')
    check_timeout = timeout-time.time()
    solv.set('timeout', int(check_timeout*1000))
    checked_solv = solv.check()
    print('...')
'''

while time.time() < timeout and solv.check()==sat:

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('Solution 1) Obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    solv.add( max_dist < best_distance)
    #solv.add( Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    
    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec
    
    #checked_solv = solv.check()
    
ending_time = time.time()
print('----------------Recap----------------------')
print('Total time: ', to_print[0])
print('Time used: ', ending_time - starting_time)
print('')
for i in range(len(to_print)-1):
    for j in range(4):
        print(to_print[i+1][j], end='')
    print('')

print('') 
print('Best solution:')   
item_pred, cour_item = [(i,j) for j in range(n_new+m) for i in range(n_new+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n_new) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz_new, cpt)
print('Minimized max distance: ', best_distance + sum(cluster_path_distances))

NameError: name 'n_new' is not defined

In [19]:
starting_time = time.time()
timeout = starting_time + 60*5
check_timeout = timeout-time.time()
print('start')
solv, max_dist, pred, v = mcp(n,m,D,sz,cpt)
print('build!', timeout-time.time())
checked_solv = solv.check()
print('a')
to_print=[check_timeout]

if checked_solv==sat:
    best_model = solv.model()
    best_distance = 1354 # best_model.evaluate(max_dist).as_long()

    to_print.append(('solution 1) obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    #solv.add( Not( And([pred[i][j] for i in range(n) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    solv.add( max_dist < best_distance)
    print('qui')
    check_timeout = timeout-time.time()
    solv.set('timeout', int(check_timeout*1000))
    checked_solv = solv.check()
    print('...')

while time.time() < timeout and checked_solv==sat:

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('Solution 1) Obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    solv.add( max_dist < best_distance)
    #solv.add( Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    
    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec
    
    checked_solv = solv.check()
    
ending_time = time.time()
print('----------------Recap----------------------')
print('Total time: ', to_print[0])
print('Time used: ', ending_time - starting_time)
print('')
for i in range(len(to_print)-1):
    for j in range(4):
        print(to_print[i+1][j], end='')
    print('')

print('') 
print('Best solution:')   
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)


start
build! 299.4046564102173
a
qui
...
----------------Recap----------------------
Total time:  300.0
Time used:  5.795121431350708

solution 1) obj = 1354; Time used: 0.6628813743591309
Solution 1) Obj = 587; Time used: 0.7214276790618896
Solution 1) Obj = 400; Time used: 0.7364282608032227
Solution 1) Obj = 368; Time used: 1.2881200313568115
Solution 1) Obj = 262; Time used: 1.5706934928894043
Solution 1) Obj = 260; Time used: 1.7077810764312744
Solution 1) Obj = 236; Time used: 1.7307825088500977
Solution 1) Obj = 226; Time used: 2.0817251205444336

Best solution:
Corriere 3: 12 -> 0 -> 12   Peso trasportato: 11 / 190
Corriere 5: 14 -> 6 -> 1 -> 14   Peso trasportato: 35 / 185
Corriere 1: 10 -> 2 -> 4 -> 10   Peso trasportato: 25 / 185
Corriere 4: 13 -> 3 -> 13   Peso trasportato: 16 / 195
Corriere 2: 11 -> 5 -> 8 -> 11   Peso trasportato: 21 / 185
Corriere 0: 9 -> 7 -> 9   Peso trasportato: 14 / 190
Minimized max distance:  226


In [18]:
starting_time = time.time()
timeout = starting_time + 60*5
check_timeout = timeout-time.time()
print('start')
solv, max_dist, cour_pred = mcp_3D(n,m,D,sz,cpt)
print('build!', timeout-time.time())
checked_solv = solv.check()
print('a')
to_print=[check_timeout]

if checked_solv==sat:
    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    to_print.append(('solution 1) obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    #solv.add( Not( And([pred[i][j] for i in range(n) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    solv.add( max_dist < best_distance)
    print('best', best_distance)
    check_timeout = timeout-time.time()
    solv.set('timeout', int(check_timeout*1000))
    checked_solv = solv.check()
    print('...')

while time.time() < timeout and checked_solv==sat:

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()
    print(best_distance)
    to_print.append(('Solution 1) Obj = ', best_distance, '; Time used: ', time.time()-starting_time))

    solv.add( max_dist < best_distance)
    #solv.add( Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])] + [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])])) )
    
    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec
    
    checked_solv = solv.check()
    
ending_time = time.time()
print('----------------Recap----------------------')
print('Total time: ', to_print[0])
print('Time used: ', ending_time - starting_time)
print('')
for i in range(len(to_print)-1):
    for j in range(4):
        print(to_print[i+1][j], end='')
    print('')

print('') 
print('Best solution:')   
#item_pred, cour_item = [(i,j) for c in range(m) for j in range(n+1) for i in range(n+1) if best_model.evaluate(cour_pred[c][i][j])], [(c, i) for j in range(n) for c in range(m) for i in range(n) if best_model.evaluate(cour_pred[c][i][j])]
#print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)


start
cour pred fattta
peso fatto
coerenza fatta
no loop fatto
max dist fatto
build! 286.51366806030273
a
best 1607
...
----------------Recap----------------------
Total time:  300.0
Time used:  300.0093939304352

solution 1) obj = 1607; Time used: 185.37963795661926

Best solution:
Minimized max distance:  1607


In [40]:
time_for_main_search=0.5 #between 0 and 1
starting_time=time.time()
timeout = starting_time + 60*5*time_for_main_search
check_timeout = timeout-time.time()

print('Time for main search:', check_timeout)
solv, max_dist, pred, v = mcp(n,m,D,sz,cpt)

first_timstamp=time.time()
checked_solv = solv.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv.model()
    best_distance = 427 #best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )
    solv.add( max_dist < best_distance)

    
    check_timeout = timeout-time.time()
    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)


while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)

    check_timeout = timeout-time.time()
    solv.set('timeout', int((timeout-time.time())*1000))

    best_model = solv.model()
    best_distance = best_model.evaluate(max_dist).as_long()
    solv.add( max_dist < best_distance)
    solv.add( Not( And( [v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    first_timstamp=time.time()
    checked_solv = solv.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

print('--------------------------------------------------')
print('End of first search. Time used: ', timeout-time.time(), '/', (starting_time + 60*5)*time_for_main_search)

#--------------------------------------------------------------------------------------------------------------------------------------------------------------

timeout = time.time() + 60*(1-time_for_main_search)*5
starting_time_2=time.time()
check_timeout = timeout - starting_time_2
print('Time for optimization search:', check_timeout)

solv_2, max_dist, pred, v = mcp(n,m,D,sz,cpt)
solv_2.add(And([v[i][j] for i in range(m) for j in range(m) if best_model.evaluate(v[i][j])]))
solv_2.add(max_dist <= best_distance)

first_timstamp=time.time()
checked_solv = solv_2.check()
second_timestamp=time.time()

print('Time used for the first search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

if checked_solv==sat:
    best_model = solv_2.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    print('Time left:', timeout-time.time(), '; Distance: ', best_distance)
    print('--------------------------------------------------')

    solv_2.add( Not(And([pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])]) ))
    solv_2.add( max_dist < best_distance)

    check_timeout = timeout-time.time()
    solv_2.set('timeout', int(check_timeout*1000))
    first_timstamp=time.time()
    checked_solv = solv_2.check()
    second_timestamp=time.time()
    print('Time used for search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)


while time.time() < timeout and checked_solv==sat:

    print('Current time left:', timeout-time.time(), '; Best distance found so far: ', best_distance)
    check_timeout = timeout-time.time()
    solv_2.set('timeout', int(check_timeout*1000))
    best_model = solv_2.model()
    best_distance = best_model.evaluate(max_dist).as_long()

    solv_2.add( max_dist < best_distance)
    solv_2.add( Not( And( [pred[i][j] for i in range(n+m) for j in range(n+m) if best_model.evaluate(pred[i][j])])) )

    first_timstamp=time.time()
    checked_solv = solv_2.check()
    second_timestamp=time.time()
    print('--------------------------------------------------')
    print('Time used for the current search:', second_timestamp-first_timstamp, '/', check_timeout, '; Result: ', checked_solv)

print('--------------------------------------------------')
print('Best Solution Found:')
    
item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if best_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if best_model.evaluate(v[i][j])]
print_items_for_each_courier(cour_item, item_pred, sz, cpt)
print('Minimized max distance: ', best_distance)

print('Time left:', timeout-time.time())


Time for main search: 150.0


NameError: name 'mcp' is not defined

In [9]:
import random

def rnd_value(cour_list, times=1):
    tot=[]
    for rip in range(times):
        dists=[]
        for cour in range(m):
            random.shuffle(cour_list[cour])
            dists.append(D[n][cour_list[cour][0]] + sum([D[cour_list[cour][i]][cour_list[cour][i+1]] for i in range(len(cour_list[cour])-1)]) + D[cour_list[cour][-1]][n])
        tot.append(max(dists))
    return sum(tot)/times


#Primo Solver: ------------------------------------------------------------------------------------

cour_solv = Solver()
v=[[Bool(f"vehicle({i})_{j}")for j in range(n+m)]for i in range(m)]

#each pack have to be carried by exactly one courrier
for j in range(n):
    cour_solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))

for j in range(m):
    for i in range(m):
        if j==i:
           cour_solv.add(v[i][n+j])
        else:
            cour_solv.add(Not(v[i][n+j]))
   
   
#Constraint peso
for c in range(m):
    cour_solv.add(Sum([If(v[c][i], 1, 0)*sz[i] for i in range(n)]) <= cpt[c])


#Prima rierca: ---------------------------------------------------------------------------------------
    
time_for_main_search=0.5 #between 0 and 1
starting_time=time.time()
timeout = starting_time + 60*5*time_for_main_search
check_timeout = timeout-time.time()

print('a')

checked_solv = cour_solv.check()

if checked_solv==sat:
    best_model = cour_solv.model()
    v_mod = [(i,j) for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])]
    cour_list=[[] for i in range(m)]
    for item in v_mod:
        cour_list[item[0]].append(item[1])
    best_d = rnd_value(cour_list) 
    #cour_solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    check_timeout = timeout-time.time()
    checked_solv = cour_solv.check()
i=0
while time.time() < timeout and checked_solv==sat:
    i+=1
    print(i, end= '-')
    current_model = cour_solv.model()
    v_mod = [(i,j) for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])]
    cour_list=[[] for i in range(m)]
    for item in v_mod:
        cour_list[item[0]].append(item[1])
    current_d = rnd_value(cour_list) 
    #cour_solv.add( Not( And([v[i][j] for i in range(m) for j in range(n) if best_model.evaluate(v[i][j])==True])) )

    if current_d < best_d:
        best_model = current_model
        best_d = current_d
        print('')
        print(best_d)

    check_timeout = timeout-time.time()
    cour_solv.set('timeout', int((timeout-time.time())*1000))
    checked_solv = cour_solv.check()
    
      

a
1-2-
1535.0
